In [30]:
import pandas as pd
import numpy as np
import os
from functions_distance import *

In [2]:
path_data = '/Users/alouette/Documents/Perceptual_Space_ALS/'

### Compute distance for each sensory modality
Euclidian distance based on 2 first dimension of PCA space 

In [3]:
stims = ['music','face','odor']
nb_stim = 18

for stim in stims:
    files_pca = [f for f in os.listdir(path_data + 'df_pca/') if f.endswith(stim+'_PCA.csv')]

    all_dist = np.zeros((nb_pairs(nb_stim),len(files_pca)))
    all_sub = []
    for i, f in enumerate(files_pca):
        df = pd.read_csv(path_data+'df_pca/'+ f, index_col=0)
        pairs = define_pairs(df.stimulus_name)
        dist = compute_all_dist(df, pairs, name_col='stimulus_name',
                           x_col='coord.Dim.1', y_col='coord.Dim.2')
        all_dist[:,i] += dist
        all_sub.append(df.subject.values[0])

    data_df = np.concatenate((np.array(pairs),all_dist),axis=1)
    df_all = pd.DataFrame(data=data_df, columns=['p0','p1']+all_sub)
    df_all.to_csv(path_data+'Python/all_dist_stim='+stim+'.csv')

### Compare perceptual space
Compare the consistency (R) btw space across subject for each sensory modality

In [50]:
from scipy import stats

def compute_all_R(df, pairs):
    """
    compute all correlations btw all pairs of data
    """
    corr = np.zeros((len(pairs)))
    for i, p in enumerate(pairs):
        a1 = np.squeeze(df[[p[0]]].values)
        a2 = np.squeeze(df[[p[1]]].values)
        r = stats.pearsonr(a1, a2)
        corr[i] += r
    return corr 

def define_pairs(names):
    """
    compute all pairs of 2 stim to compare
    """
    pairs = [(p0, p1) for p0, p1 in combinations(names,2)]
    return pairs


In [52]:
path_df = path_data+'Python/all_dist_stim={}.csv'
stim = 'music'

df = pd.read_csv(path_df.format(stim), index_col=0)
subj = [c for c in df.columns if c not in ['p0','p1']]
pairs_su = define_pairs(subj)
corr = compute_all_R(df, pairs)
print(corr)


KeyError: "None of [Int64Index([1], dtype='int64')] are in the [columns]"